In [1]:
import pandas as pd
import random
from llama_index.core.prompts import PromptTemplate
from resourses import *
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

c:\Users\khang\anaconda3\envs\RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
def displayify_df(df):
    """For pretty displaying DataFrame in a notebook."""
    display_df = df.style.set_properties(
        **{
            "inline-size": "300px",
            "overflow-wrap": "break-word",
        }
    )
    display(display_df)

# Evaluate RAG 

## 1. Load data

In [27]:
df = pd.read_csv("qna_dataset")
#random.seed(42)
random.seed(1)
test_queries = random.sample(list(df["question"]), 100)
print(len(test_queries))
test_queries[:3]

100


['Chồng đánh vợ nhập viện thì người vợ có quyền yêu cầu ly hôn không?',
 'Phơi lúa trên đường bộ bị phạt bao nhiêu tiền?',
 'Chồng chết thì quan hệ hôn nhân có chấm dứt không? ']

### Get answer from llm

In [24]:
#  baseline model
llm = get_llm(local=False)
# rag model
query_engine = get_query_engine(llm=llm)

In [ ]:
# Lay 10 cai 1 luc 
baseline_answer = []
rag_answer = []
for query in test_queries:
    baseline_answer.append(llm.complete(query))
    rag_answer.append(query_engine.query(query))


In [31]:
len(baseline_answer), len(rag_answer)
eval_df = pd.DataFrame({"query": test_queries, "baseline_answer": baseline_answer, "rag_answer": rag_answer})
eval_df.to_csv("gemini_eval_df_100.csv", index=False)


## Answer Relevant Score

### Load Dataset

Evaluates the relevancy of response to a query. This evaluator considers the query string and response string.


In [7]:
import pandas as pd
eval_df = pd.read_csv("eval_df_100.csv")

queries = eval_df["query"].tolist()
baseline_answers = eval_df["baseline_answer"].tolist()
rag_answers = eval_df["rag_answer"].tolist()

In [4]:
eval_df.head()

,query,baseline_answer,rag_answer
0,Xin hỏi nếu vợ bị tâm thần thì khi bán mảnh đấ...,"Tất nhiên, tôi sẽ giúp bạn giải đáp các vấn đề...","Dựa trên thông tin bạn cung cấp, tôi xin trả l..."
1,Cho tôi hỏi thời kỳ kết hôn được hiểu như thế ...,Thời kỳ kết hôn là một khoảng thời gian trong ...,Thời kỳ kết hôn được hiểu là từ ngày khi vợ ho...
2,Xe ô tô bị mất trộm có phải đóng phí sử dụng đ...,Xin lỗi vì sự nhầm lẫn nhưng tôi không thể cun...,"Xin lỗi, nhưng tôi cần hiểu rõ hơn về câu hỏi ..."
3,Nhiệm vụ của tiếp viên hàng không được quy địn...,"Tôi xin lỗi, tôi không có khả năng phân tích h...","Xin lỗi, tôi không thể hỗ trợ người dùng câu h..."
4,Cơ quan nào có thẩm quyền cấp giấy phép lái xe?,Lý do của việc cấp Giấy chứng nhận lái xe phải...,Lý do cơ sở pháp lý được đề cập trong văn bản:...


### Evaluate

In [8]:
from llama_index.core.prompts import ChatPromptTemplate, ChatMessage, MessageRole
DEFAULT_SYSTEM_TEMPLATE_VI = """
Bạn là một hệ thống đánh giá chuyên nghiệp cho một chatbot trả lời câu hỏi.

Bạn sẽ được cung cấp các thông tin sau:
- một câu hỏi từ người dùng, và
- một câu trả lời do hệ thống tạo ra

Bạn cũng có thể được cung cấp một câu trả lời tham khảo để so sánh trong quá trình đánh giá.

Nhiệm vụ của bạn là đánh giá mức độ liên quan và tính chính xác của câu trả lời do hệ thống tạo ra.
Kết quả của bạn phải bao gồm chính xác 2 dòng.
Dòng 1 là số điểm bạn chấm.
Dòng 2 là lý do ngắn gọn bạn chấm điểm.

Hãy tuân thủ các nguyên tắc sau khi chấm điểm:
- Điểm số phải nằm trong khoảng từ 1 đến 5, trong đó 1 là tệ nhất và 5 là tốt nhất.
- Nếu câu trả lời không liên quan đến câu hỏi, hãy cho điểm **1**.
- Nếu câu trả lời có liên quan nhưng có lỗi sai, hãy cho điểm **từ 2 đến 3**.
- Nếu câu trả lời vừa liên quan vừa chính xác, hãy cho điểm **từ 4 đến 5**.

### Ví dụ phản hồi:
4.0  
Câu trả lời do hệ thống tạo ra có cùng thông tin với câu trả lời tham khảo, nhưng chưa súc tích bằng.
"""

DEFAULT_USER_TEMPLATE_VI = """
## Câu hỏi của người dùng
{query}

## Câu trả lời tham khảo
{reference_answer}

## Câu trả lời do hệ thống tạo ra
{generated_answer}
"""

DEFAULT_EVAL_TEMPLATE_VI = ChatPromptTemplate(
    message_templates=[
        ChatMessage(role=MessageRole.SYSTEM, content=DEFAULT_SYSTEM_TEMPLATE_VI),
        ChatMessage(role=MessageRole.USER, content=DEFAULT_USER_TEMPLATE_VI),
    ]
)

In [9]:
from llama_index.core.evaluation import CorrectnessEvaluator

llm = get_llm(local=False)
query_engine = get_query_engine(llm=llm)

evaluator = CorrectnessEvaluator(
    llm=llm,
    eval_template=DEFAULT_EVAL_TEMPLATE_VI,
)

EXAMPLE

In [8]:
# sample_query = queries[1]
# response = baseline_answers[1]
# eval_result = evaluator.evaluate(query=sample_query, response=response)
# eval_result.score, eval_result.feedback

In [22]:
def evaluate_correctness(queries, answers):
    total_score = 0
    total_pass = 0
    num_invalid = 0
    evaluator = CorrectnessEvaluator(
        llm=llm,
        eval_template=DEFAULT_EVAL_TEMPLATE_VI,
    )
    for query, answer in zip(queries, answers):
        try:
            eval_result = evaluator.evaluate(query=query, response=answer)
            print(query, answer)
            print(eval_result.score, eval_result.passing, eval_result.feedback)
            if isinstance(eval_result.score, float):
                total_score += eval_result.score
            if isinstance(eval_result.passing, bool) and eval_result.passing:
                total_pass += 1
        except Exception as e:
            print(e)
            num_invalid += 1
    return total_score, total_pass , len(queries) - num_invalid



In [21]:
queries[:2], baseline_answers[:2]

(['Xin hỏi nếu vợ bị tâm thần thì khi bán mảnh đất thì tôi có cần chữ kí của vợ không? Thủ tục chuyển nhượng được thực hiện như thế nào?',
  'Cho tôi hỏi thời kỳ kết hôn được hiểu như thế nào?'],
 ['Tất nhiên, tôi sẽ giúp bạn giải đáp các vấn đề này:\n\n1. Trả cho vợ: Nếu bạn muốn giao đất cho vợ, chỉ cần ký tên vào bản hợp đồng hoặc giấy chứng nhận. Điều quan trọng là phải có sự minh bạch và chính xác về tình trạng tài sản của bạn.\n\n2. Quyết toán: Để thực hiện chuyển nhượng quyền lợi, bạn cần phải có số liệu chi tiết về tiền thuế và tiền thuê đất. Trong trường hợp vợ bị tâm thần, bạn nên kiểm tra các thông tin này để đảm bảo rằng mọi thứ đúng đắn.\n\n3. Thủ tục chuyển nhượng: Tuy nhiên, tôi khuyên bạn nên咨询 một luật gia hoặc người quản lý tài sản để nắm bắt chính xác điều kiện mới nhất và đầy đủ.\n\nVì vậy, nếu bạn muốn giao đất cho vợ, chỉ cần ký tên vào bản hợp đồng hoặc giấy chứng nhận. Nếu bạn có bất kỳ thắc mắc nào khác, hãy tiếp tục liên hệ với tôi để được giải thích chi tiết 

In [23]:
total_score, total_pass, valid = evaluate_correctness(queries[:2], baseline_answers[:2])
print(total_score, total_pass, valid)


Xin hỏi nếu vợ bị tâm thần thì khi bán mảnh đất thì tôi có cần chữ kí của vợ không? Thủ tục chuyển nhượng được thực hiện như thế nào? Tất nhiên, tôi sẽ giúp bạn giải đáp các vấn đề này:

1. Trả cho vợ: Nếu bạn muốn giao đất cho vợ, chỉ cần ký tên vào bản hợp đồng hoặc giấy chứng nhận. Điều quan trọng là phải có sự minh bạch và chính xác về tình trạng tài sản của bạn.

2. Quyết toán: Để thực hiện chuyển nhượng quyền lợi, bạn cần phải có số liệu chi tiết về tiền thuế và tiền thuê đất. Trong trường hợp vợ bị tâm thần, bạn nên kiểm tra các thông tin này để đảm bảo rằng mọi thứ đúng đắn.

3. Thủ tục chuyển nhượng: Tuy nhiên, tôi khuyên bạn nên咨询 một luật gia hoặc người quản lý tài sản để nắm bắt chính xác điều kiện mới nhất và đầy đủ.

Vì vậy, nếu bạn muốn giao đất cho vợ, chỉ cần ký tên vào bản hợp đồng hoặc giấy chứng nhận. Nếu bạn có bất kỳ thắc mắc nào khác, hãy tiếp tục liên hệ với tôi để được giải thích chi tiết hơn nhé!
1.0 False Câu trả lời hoàn toàn không liên quan đến câu hỏi.

Ch

- Lần 1: 
BASELINE: 100 câu - 266 điểm - 50 câu đạt
RAG: 100 câu - 392 điểm - 76 câu đạt

